## Embeddings using Last Hidden State

In [ ]:
# !pip install transformers
# !pip install transformers[sentencepiece]
# !pip install transformers[torch]

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# drive.mount('/gdrive')
%cd /content/drive/MyDrive/github/filter_data

Mounted at /content/drive
/content/drive/MyDrive/github/filter_data


In [2]:
import os
import pandas as pd
from datetime import datetime
import pickle
from transformers import pipeline

In [47]:
import torch
def get_embeddings(text,
                   model_name = 'ai4bharat/IndicBERTv2-MLM-only',
                   task = "feature-extraction"):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  if task == "feature-extraction":
    extractor = pipeline(model=model_name, task=task, device=device)
    embeddings = extractor(text, return_tensors=True)
  elif task == "hidden_states":
    tbd
  else:
    print(f'Task either not specified or not supported.'
          f'Using default task: {task} and model: {model_name}')
    get_embeddings(text)
  return embeddings

In [61]:
lang = 'en'
read_dir = os.path.join(os.getcwd(),'formatted_data', lang)
save_dir = os.path.join(os.getcwd(),'embedding_data', lang)
data_files = os.listdir(read_dir)
data_files = [file for file in data_files if file.endswith(".csv")]
len(data_files)

15

In [62]:
for idx, file_name in enumerate(data_files, start=1):
  df = pd.read_csv(f'{os.path.join(read_dir, file_name)}')
  df = df.drop_duplicates()
  df['sentence'] = df['sentence'].apply(lambda x: x.lower())
  embeddings = get_embeddings(df.sentence.tolist())
  embeddings = [embedding.mean(dim=1).flatten() for embedding in embeddings]
  df['embeddings'] = embeddings

  #save embeddings
  now = datetime.now()
  intent = file_name.split(".")[0]
  save_file_name = f'{intent}_{now.strftime("%d-%m_%H:%M")}'
  # print(save_file_name)

  with open(f'{os.path.join(save_dir, save_file_name)}.pickle', 'wb') as f:
    pickle.dump(df, f)
  print(f'{idx}: embeddings saved for intent "{intent}" at location: {save_dir}')
print('--- All Done ---')

1: embeddings saved for intent "mobile_recharge_prepaid" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
2: embeddings saved for intent "fastag_recharge" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
3: embeddings saved for intent "confirm" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
4: embeddings saved for intent "emi_collection_full" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
5: embeddings saved for intent "mobile_recharge_postpaid" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
6: embeddings saved for intent "electricity_payment" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
7: embeddings saved for intent "gas_payment" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
8: embeddings saved for intent "balance_check" at location: /content/drive/MyDrive/github/filter_data/embedding_data/en
9: embeddings

In [ ]:
# get embeddings
MODEL = 'ai4bharat/IndicBERTv2-MLM-only'
MODEL2 = 'ai4bharat/IndicBERTv2-SS'

for key in data_dict:
  print(f'working with "{key}" intent')
  df = data_dict[key]
  print(f'original shape {df.shape}')
  df = df.drop_duplicates()
  print('after de-duplication', df.shape)
  df['sentence'] = df['sentence'].apply(lambda x: x.lower())
  df["embedding"] = df['sentence'].apply(lambda x: get_embeddings(x, model_name=MODEL))
  data_dict[key] = df

confirm
fastag_recharge
emi_collection_partial
emi_collection_full
cancel
balance_check
p2p_transfer
electricity_payment
gas_payment
mobile_recharge_prepaid
mobile_recharge_postpaid
inform
petrol_payment
upi_creation
insurance_renewal


(4177, 4)

In [ ]:
# from transformers import AutoTokenizer, BertModel, BertConfig
# import torch

# config = BertConfig.from_pretrained(MODEL2, output_hidden_states=True)
# model = BertModel.from_pretrained(MODEL2, config=config)
# tokenizer = AutoTokenizer.from_pretrained(MODEL2)

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertModel were not initialized from the model checkpoint at ai4bharat/IndicBERTv2-SS and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
